In [1]:
import pandas as pd
import numpy as np

# Read taxonomy table

In [36]:
df_tax = pd.read_csv('taxonomy.tsv', sep='\t')
taxonomy = []
for idx in df_tax.index:
    taxon = df_tax.loc[idx,'Taxon']
    taxon = taxon.split(';')
    res = [df_tax.loc[idx,'Feature ID']]
    lowest_classified_taxon = 'unclassified'
    for i in np.arange(7):
        if i < len(taxon):
            res.append(taxon[i])
            if 'unclassified' not in taxon[i] and 'uncultured' not in taxon[i] and 'unidentified' not in taxon[i]:
                lowest_classified_taxon = taxon[i]
        else:
            res.append('unclassified')
    res.append(lowest_classified_taxon)
    taxonomy.append(res)
    
df_tax = pd.merge(df_tax, pd.DataFrame(taxonomy, columns=['Feature ID','Kingdom','Phylum','Class','Order','Family','Genus','Species','LowestClassifiedTaxon']), left_on='Feature ID', right_on='Feature ID', how='left')
df_tax = df_tax[['Feature ID','Confidence','Kingdom','Phylum','Class','Order','Family','Genus','Species','LowestClassifiedTaxon']]
df_tax = df_tax.rename({'Feature ID':'ASV'}, axis=1).set_index('ASV')

# remove genus tag in species
df_tax['Species'] = [y.replace(x+'-','') for x,y in zip(df_tax['Genus'], df_tax['Species'])]
#df_tax['LowestTaxon'] = [y.replace(x+'-','') for x,y in zip(df_tax['Genus'], df_tax['LowestTaxon'])]

# # select only bacteria
# df_tax = df_tax[df_tax.Kingdom=='Bacteria']

df_tax.head()

,Confidence,Kingdom,Phylum,Class,Order,Family,Genus,Species,LowestClassifiedTaxon
ASV,,,,,,,,,
38f9479d2f37d1036274704b09d2e3c5,0.999064,Bacteria,Firmicutes,Clostridia,Lachnospirales,Lachnospiraceae,Blautia,unclassified,Blautia
ffa2d3d4e030a8f45850c379c0d196f5,1.000000,Bacteria,Actinobacteriota,Actinobacteria,Bifidobacteriales,Bifidobacteriaceae,Bifidobacterium,unclassified,Bifidobacterium
f7a1c2a7dc74eb24446faaa79f519ee6,0.991931,Bacteria,Firmicutes,Clostridia,Lachnospirales,Lachnospiraceae,Agathobacter,unclassified,Agathobacter
0ae1308cf1201de65c4edfcd616e89b2,0.715171,Bacteria,Firmicutes,Clostridia,Lachnospirales,Lachnospiraceae,Fusicatenibacter,uncultured-organism,Fusicatenibacter
8a2950244650b024368aa8174145ba33,0.998812,Bacteria,Firmicutes,Clostridia,Lachnospirales,Lachnospiraceae,Blautia,unclassified,Blautia


# Read metadata

In [47]:
df_meta = pd.read_csv('metadata.txt', sep='\t').set_index('#SampleID')
df_meta.head()

,Collection_Date,env_material,ETHNICITY,Host_Age,host_body_mass_index,Host_diet,host_sex,Group,Subject_ID,status,Replicate,Sample_Name,SAMPLE_TYPE
#SampleID,,,,,,,,,,,,,
SRR6185224,16-Apr,Feces,NZ European,52,20.6,High_dietary_fibre_intake,female,Prebiotic,ADAPT03,pre,ADAPT03 sample 1,ADAPT03_STP1,Pre-intervention
SRR6185259,16-Apr,Feces,NZ European,52,20.6,High_dietary_fibre_intake,female,Prebiotic,ADAPT03,post,ADAPT03 sample 2,ADAPT03_ETP1,Post-intervention
SRR6185212,16-May,Feces,NZ European,52,20.6,High_dietary_fibre_intake,female,Placebo,ADAPT03,pre,ADAPT03 sample 3,ADAPT03_STP2,Pre-intervention
SRR6185177,16-Jun,Feces,NZ European,52,20.6,High_dietary_fibre_intake,female,Placebo,ADAPT03,post,ADAPT03 sample 4,ADAPT03_ETP2,Post-intervention
SRR6185278,16-Apr,Feces,Chinese,52,22.3,High_dietary_fibre_intake,female,Placebo,ADAPT04,pre,ADAPT04 sample 1,ADAPT04_STP1,Pre-intervention


# Prepare relative abundance table

In [58]:
df_count = pd.read_csv('otu.txt', sep='\t')
df_count = df_count.rename({'#OTU ID':'ASV'}, axis=1).set_index('ASV')
df_count = df_count[list(df_meta.index)]
df_count = df_count.div(df_count.sum(axis=0), axis=1)
df_count = df_count[(df_count.T != 0).any()]

# merge tax and absolute abundance
df_count = pd.merge(df_tax[['LowestClassifiedTaxon']], df_count, left_index=True, right_index=True, how='inner').set_index('LowestClassifiedTaxon', drop=True)

# merge index (row sum for the same index)
df_count = df_count.groupby(df_count.index).sum()

df_count.head()

,SRR6185224,SRR6185259,SRR6185212,SRR6185177,SRR6185278,SRR6185262,SRR6185192,SRR6185178,SRR6185236,SRR6185264,...,SRR6185199,SRR6185164,SRR6185258,SRR6185213,SRR6185253,SRR6185162,SRR6185257,SRR6185214,SRR6185179,SRR6185161
LowestClassifiedTaxon,,,,,,,,,,,,,,,,,,,,,
Abiotrophia,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
Acetanaerobacterium,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
Acidaminococcus,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000182,0.000000,0.008199,0.0,0.0,0.0,0.0,0.022204,0.001361,0.004546
Actinidia-chinensis,0.0,0.0,0.0,0.000279,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
Actinomyces,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000114,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000


In [59]:
df_count_T = df_count.T
df_count_T.index.name = 'SampleID'

In [60]:
df_count_T.to_excel('16S_relative_abundance.xlsx')